In [29]:
from datasets import Dataset
import pandas as pd

def parse_conll_file(file_path):
    tokens = []
    ner_tags = []
    current_tokens = []
    current_tags = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:  # Empty line indicates end of a sample
                if current_tokens:
                    tokens.append(current_tokens)
                    ner_tags.append(current_tags)
                    current_tokens = []
                    current_tags = []
                continue
                
            parts = line.split('\t')
            if len(parts) >= 2:
                token = parts[0]
                tag = parts[1]
                current_tokens.append(token)
                current_tags.append(tag)
    
    # Add last sample if exists
    if current_tokens:
        tokens.append(current_tokens)
        ner_tags.append(current_tags)
        
    return tokens, ner_tags

# Parse your file
file_path = "../output copy.txt"
tokens, ner_tags = parse_conll_file(file_path)

# Create label mappings
label_list = sorted(set(tag for tags in ner_tags for tag in tags))
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

# Convert tags to IDs
ner_tags_ids = [[label2id[tag] for tag in tags] for tags in ner_tags]

# Create Hugging Face Dataset
dataset = Dataset.from_dict({
    "tokens": tokens,
    "ner_tags": ner_tags_ids
})
print(dataset['tokens'])
print(dataset['ner_tags'])

[['3pcs', 'Bottle', 'Stopper', 'በማንኛውም', 'ጠርሙስ', 'ጫፍ', 'የሚገጠም', 'ለዘይት', 'እና', 'መሰል', 'ነገሮች', 'መቀነሻ', 'የሚሆን', 'በአግባቡ', 'እየመጠንን', 'ለመጠቀም', 'ተመራጭ', 'ዋጋ', '3ፍሬ', '400', 'ብር', 'ውስን', 'ፍሬ', 'ነው', 'ያለው', 'አድራሻ', 'መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ', 'ቢሮ', 'ቁ', 'S05S06', '0902660722', '0928460606', 'በTelegram', 'ለማዘዝ', 'ይጠቀሙ', 'zemencallcenter', 'zemenexpressadmin', 'ለተጨማሪ', 'ማብራሪያ', 'የቴሌግራም', 'ገፃችን', 'httpstelegrammezemenexpress'], ['3pcs', 'Bottle', 'Stopper', 'በማንኛውም', 'ጠርሙስ', 'ጫፍ', 'የሚገጠም', 'ለዘይት', 'እና', 'መሰል', 'ነገሮች', 'መቀነሻ', 'የሚሆን', 'በአግባቡ', 'እየመጠንን', 'ለመጠቀም', 'ተመራጭ', 'ዋጋ', '3ፍሬ', '400', 'ብር', 'ውስን', 'ፍሬ', 'ነው', 'ያለው', 'አድራሻ', 'መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ', 'ቢሮ', 'ቁ', 'S05S06', '0902660722', '0928460606', 'በTelegram', 'ለማዘዝ', 'ይጠቀሙ', 'zemencallcenter', 'zemenexpressadmin', 'ለተጨማሪ', 'ማብራሪያ', 'የቴሌግራም', 'ገፃችን', 'httpstelegrammezemenexpress'], ['1', 'pairs', 'Sneaker', 'Crease', 'Protector', 'ዋጋ', '400', 'ብር', 'ውስን', 'ፍሬ', 'ነው', 'ያለው', 'አድራሻ', 'መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ', 'ቢሮ', 'ቁ', 'S05S06', '0902660722'

In [45]:
label_list=['B-Product','I-Product','B-LOC','I-LOC','B-PRICE','I-PRICE','O']
label2id={}
id2label={}
for idx,label in enumerate(label_list):
    label2id[label]=idx
    id2label[idx]=label
# print(label2id)
ner_tags_ids=[]
# print(ner_tags)
ner_tags_ids = [[label2id[tag] for tag in tags] for tags in ner_tags]
print(ner_tags_ids)
dataset=Dataset.from_dict({
    "tokens":tokens,
    "ner_tags":ner_tags_ids}
)

 

[[0, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 5, 5, 6, 6, 6, 6, 6, 2, 3, 3, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6], [0, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 5, 5, 6, 6, 6, 6, 6, 2, 3, 3, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6], [0, 1, 1, 1, 1, 4, 5, 5, 6, 6, 6, 6, 2, 3, 3, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]]


split to trian and test dataset 

In [37]:
dataset = dataset.train_test_split(test_size=0.2, seed=42)

AttributeError: 'DatasetDict' object has no attribute 'train_test_split'

In [43]:
train_data=dataset.train

AttributeError: 'DatasetDict' object has no attribute 'train'

In [ ]:
from transformers import AutoTokenizer

model_id = "Davlan/afro-xlmr-base"  # African language-optimized model
tokenizer = AutoTokenizer.from_pretrained(model_id, add_prefix_space=True)
tokenizer(dataset['Tokens'],is_split_to_words=True)

ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /api/models/Davlan/afro-xlmr-base/tree/main/additional_chat_templates?recursive=False&expand=False (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000201CC28E210>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 0b9a64a3-225e-4955-8a8e-168d2f90be78)')

In [46]:
labels=[]

for i,label in enumerate(dataset['ner_tags']):
    print(label)

[0, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 5, 5, 6, 6, 6, 6, 6, 2, 3, 3, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
[0, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 5, 5, 6, 6, 6, 6, 6, 2, 3, 3, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
[0, 1, 1, 1, 1, 4, 5, 5, 6, 6, 6, 6, 2, 3, 3, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
